In [1]:
import sys
import os
import shutil
import glob
import random
import math
from numpy import random
import numpy as np
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy
import pandas
import sklearn
from collections import Counter
from matplotlib.font_manager import FontProperties
from matplotlib import ticker
from pymoo.core.problem import ElementwiseProblem
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.decomposition.asf import ASF
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.termination import get_termination
from pymoo.decomposition.asf import ASF
from pymoo.optimize import minimize
import matplotlib.patches as mpatches
import time
from matplotlib.pyplot import MultipleLocator
from itertools import product
font = FontProperties(fname=r"c:\windows\fonts\simhei.ttf", size=14)
fontlegend = FontProperties(fname=r"c:\windows\fonts\simhei.ttf", size=14)
print(sys.version)
print('numba version: {}'.format(np.__version__))
print('flopy version: {}'.format(flopy.__version__))

3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
numba version: 1.23.4
flopy version: 3.3.4


In [2]:
time=[]
with open(r'D:\\工作需\\R程序计算\\R程序翻译\\timeC13.txt', 'r+') as obj:
    for line in obj.readlines():
        time.append(line)
    time = [line.strip("\n") for line in time]
Tlist=np.array(time)

In [3]:
'''
read mppth file and save IJK
'''
def save_mp_resnb(pth_file, out_csv, tar_colnames=None, add_ij_info=False, in_model=None):
    if tar_colnames == None:
        tar_colnames = ["particleid", "x", "y", "z","time", "k"]
    pthobj = flopy.utils.PathlineFile(pth_file)
    df_all = pandas.DataFrame(pthobj._data)[tar_colnames]
    if add_ij_info:
        i,j = in_model.dis.get_rc_from_node_coordinates(df_all["x"].values, df_all["y"].values)
        df_all.insert(len(df_all.columns), "i", i)
        df_all.insert(len(df_all.columns), "j", j)
    df_all.to_csv(out_csv, index=None)
    return "Complete: %s"%(pth_file)

# CELL.txt

In [4]:
workspace = os.path.join('.')
def mpthcell(allname, modelname, welnum, hk):
    nm=modelname
    is_silent = True
    in_mfn='D:/工作需/G-case/%s_MODFLOW_text/%s.mfn'%(allname,allname) #C13-test_MODFLOW_text path
    m = flopy.modflow.Modflow.load(in_mfn, model_ws=workspace, version="mf2005", exe_name = 'mf2005.exe')
    nrow, ncol, nlay, nper = m.nrow_ncol_nlay_nper
    m.write_input()
    m.run_model(silent=is_silent)
    
    #particle in each cell
    plocs = []
    pids = []
    ids=0
    for kdx, idx, jdx in product(range(nlay),range(nrow),range(ncol)):
        plocs.append((kdx, idx, jdx))
        ids+=1
        pids.append(ids)
    part0 = flopy.modpath.ParticleData(
        plocs, drape=0, structured=True, particleids=pids
    )
    pg0 = flopy.modpath.ParticleGroup(
        particlegroupname="PG1", particledata=part0, filename="ex01a.pg1.sloc"
    )
    particlegroups = [pg0]
    import time
    for idx,i in enumerate(Tlist):
        s = time.time()
        # create modpath files
        exe_name = "mp7"
        mp = flopy.modpath.Modpath7(
            modelname=nm + "_mp", flowmodel=m, exe_name=exe_name, model_ws=workspace, budgetfilename="%s.ccf"%allname
        )
        mpbas = flopy.modpath.Modpath7Bas(mp)
        mpsim = flopy.modpath.Modpath7Sim(
            mp,
            simulationtype="pathline",
            trackingdirection="forward",
            weaksinkoption="pass_through",
            weaksourceoption="pass_through",
            budgetoutputoption="summary",
            referencetime=[0, 0, 0.0],
            stoptimeoption="specified",
            stoptime=int(i)-1,
            zonedataoption="off",
            particlegroups=particlegroups,
        )

        # write modpath datasets
        mp.write_input()
        # run modpath
        mp.run_model(silent=True)
        fpth_file = workspace+"/"+nm+"_mp.mppth"
        out_csv = workspace+"/modpath_midresult.csv"
        save_mp_resnb(fpth_file, out_csv, add_ij_info=True, in_model=m)

        df=pandas.read_csv(out_csv)
        c=df.values.tolist()
        carr=np.array(c, dtype='object')
        rows, columns = carr.shape
        columns=['Particle_Index', 'X', 'Y', 'Z', 'Time', 'Cell_K', 'Cell_I', 'Cell_J']
        for i,j in product(range(rows),range(5,8)): 
            carr[i,j]=int(carr[i,j]+1)
            carr[i,0]=int(carr[i,0])
        carr=np.insert(carr, 0, values=columns, axis=0) 
        '''save path'''
        txtc=[]
        for i in Tlist:
            txtcell = os.path.join(r"D:\工作需\R程序计算\R程序翻译\%s"%nm, r"cell_%s.txt"%i) #path to save cell.txt
            txtc.append(txtcell)
        wstxt=r"D:/工作需/R程序计算/R程序翻译/"+nm
        if not os.path.exists(wstxt):
            os.makedirs(wstxt)
        with open(txtc[idx], 'w') as f: #保存至指定文件
            np.savetxt(f, carr, fmt="%s", delimiter="    ")
        e = time.time()
        print("%s/%s | %s completed in %.2fs"%(idx+1, len(Tlist), txtc[idx], e-s))
mpthcell(allname='C13-test', modelname='C13-test', welnum=1, hk=0.05)

# IN.txt

In [5]:
inname="in_1800.txt"

In [21]:
def mpthin(ptcl,i,inname):
    workspace = os.path.join('.')
    #定义模型名称和存储文件名
    nm='C13-test'
    is_silent = True  
    in_mfn=r'D:/工作需/G-case/C13-test_MODFLOW_text/C13-test.mfn'#C13-test_MODFLOW_text path
    m = flopy.modflow.Modflow.load(in_mfn, model_ws=workspace, version="mf2005", exe_name = 'mf2005.exe')
    nrow, ncol, nlay, nper = m.nrow_ncol_nlay_nper
    m.write_input()
    m.run_model(silent=is_silent)
    #Add particles to all injection wells
    data = pandas.read_table(r"D:\工作需\R程序计算\R程序翻译\井坐标文件\注井坐标C13.txt", sep="\\s+").values.tolist()#well file path
    In_wel=np.array(data)
    inw=In_wel[i:i+2,:]
    plocs = []
    pids = []
    for i in inw:
        plocs.append((int(i[2])-1,int(i[0])-1,int(i[1])-1))
    Plocs=np.concatenate([plocs, plocs], axis=1)
    cd = flopy.modpath.CellDataType(drape=0,
                                   columncelldivisions=ptcl[0],
                                   rowcelldivisions=ptcl[1],
                                   layercelldivisions=ptcl[2])
    p = flopy.modpath.LRCParticleData([cd], [Plocs])

    pg1 = flopy.modpath.ParticleGroupLRCTemplate(particlegroupname="PG2",
                                            particledata=p,
                                            filename="ex01a.pg2.sloc")
    particlegroupin = [pg1] 
    for idx,i in enumerate(Tlist):
        s = time.time()
        # create modpath files
        exe_name = "mp7"
        mp = flopy.modpath.Modpath7(
            modelname=nm + "_mp", flowmodel=m, exe_name=exe_name, model_ws=workspace, budgetfilename="C13-test.ccf"
        )
        mpbas = flopy.modpath.Modpath7Bas(mp)
        mpsim = flopy.modpath.Modpath7Sim(
            mp,
            simulationtype="pathline",
            trackingdirection="forward",
            weaksinkoption="pass_through",
            weaksourceoption="pass_through",
            budgetoutputoption="summary",
            referencetime=[0, 0, 0.0], 
            stoptimeoption="specified",
            stoptime=int(i)-1,
            zonedataoption="off",
            particlegroups=particlegroupin,
        )

        # write modpath datasets
        mp.write_input()
        # run modpath
        mp.run_model(silent=True)
        fpth_file = workspace+"/"+nm+"_mp.mppth"
        out_csv = workspace+"/modpath_midresult.csv"
        save_mp_resnb(fpth_file, out_csv, add_ij_info=True, in_model=m)
        df=pandas.read_csv(out_csv) 
        c=df.values.tolist()
        carr=np.array(c, dtype='object')
        rows, columns = carr.shape
        columns=['Particle_Index', 'X', 'Y', 'Z', 'Time', 'Cell_K', 'Cell_I', 'Cell_J']
        for i,j in product(range(rows),range(5,8)): 
            carr[i,j]=int(carr[i,j]+1)
            carr[i,0]=int(carr[i,0])
        carr=np.insert(carr, 0, values=columns, axis=0)
        '''save path'''
        txtc=[]
        for i in Tlist:
            txtcell = os.path.join(r"D:\工作需\R程序计算\R程序翻译\%s"%nm, inname)#path to save cell.txt
            txtc.append(txtcell)
        wstxt=r"D:/工作需/R程序计算/R程序翻译/"+nm
        if not os.path.exists(wstxt):
            os.makedirs(wstxt)
        with open(txtc[idx], 'w') as f: #保存至指定文件
            np.savetxt(f, carr, fmt="%s", delimiter="    ")
        e = time.time()

In [22]:
#injection wells file path
wel = pandas.read_table(r'D:\工作需\R程序计算\R程序翻译\井坐标文件\注井坐标C13.txt', sep="\\s+").values.tolist()
welarr = np.array(wel)

# calculation of the characteristic volume of in-situ uranium leaching

In [23]:
dir_path=r"D:/工作需/R程序计算/R程序翻译" 
dir_path2=r"D:/工作需/R程序计算/R程序翻译/C13-test" #cel.txt and in,txt path
timezlh=dir_path+"/timeC13.txt" #time file

zlc_dir=dir_path+"/总流场"
dylc_dir=dir_path+"/单元流场"

jzb_dir=dir_path+"/井坐标文件" 

jzb_file_lis=os.listdir(jzb_dir)
if jzb_file_lis[0].startswith("抽井"):
    cj_path=jzb_dir+"/"+jzb_file_lis[0]
    zj_path=jzb_dir+"/"+jzb_file_lis[1]
else:
    cj_path = jzb_dir + "/" + jzb_file_lis[1]
    zj_path = jzb_dir + "/" + jzb_file_lis[0]
    
def intjudge(x):
    if  isinstance(x, int)==True:
        index=-1
    else:
        index=1
    return index
    
def newly_build_dir():
    """ Check whether the save path exists, create it if it does not exist"""
    for i in (zlc_dir,dylc_dir):
        if not os.path.exists(i):
            os.mkdir(i)

def get_file_list(inname):
    """ Get a list of files to process"""
    with open(timezlh,"r",encoding="utf-8") as f:
        l=f.read().strip().split("\n")
    file_list=[(i,dir_path2+"/"+"cell_"+i+".txt",dir_path2+"/"+inname) for i in l]
    return file_list

def get_cj_ijk():
    """ ijk of pumping wells"""
    data=pandas.read_csv(cj_path,sep="\\s+",usecols=["I","J","K"]).values.tolist()
    set_data=set([tuple(i) for i in data]) 
    return set_data

def get_zj_ijk(file_path):
    """ ijk of injection wells"""
    dic={}
    data = pandas.read_table(zj_path, sep="\\s+").values.tolist()
    for i in data:
        dic.setdefault(i[-1],set()).add(tuple(i[:3]))
    for i in dic:
        dic[i]=tuple(dic[i])
    dic1={}
    data = pandas.read_table(file_path, sep="\\s+", usecols=["Particle_Index","Time", "Cell_K", "Cell_I", "Cell_J"])
    data=data[["Particle_Index","Time", "Cell_I", "Cell_J", "Cell_K"]]
    data["ijk"]=data.apply(lambda x:(x["Cell_I"],x["Cell_J"],x["Cell_K"]),axis=1)
    for k,v in dic.items():
        dic1[k]=set()
        all_index=[]
        for i in v:
            p=data[(data["ijk"]==i) & (data["Time"]==0)]
            l=p["Particle_Index"].values.tolist()
            all_index+=l
        data1=data[data["Particle_Index"].isin(all_index)][["Cell_I", "Cell_J", "Cell_K"]].values.tolist()
        for i in data1:
            dic1[k].add(tuple(i))
    return dic1

def get_set_A(file_path):
    """ get setA"""
    cj_ijk=get_cj_ijk()
    data = pandas.read_table(file_path,sep="\\s+", usecols=["Particle_Index","Time","Cell_K","Cell_I","Cell_J"])
    data["ijk"] = data.apply(lambda x: (x["Cell_I"], x["Cell_J"], x["Cell_K"]), axis=1)
    data1=data[data["ijk"].isin(cj_ijk)]["Particle_Index"].values.tolist()
    d=tuple(set(data1))
    data=data[(data["Time"]==0) & (data["Particle_Index"].isin(d))][["Cell_I","Cell_J","Cell_K"]].values.tolist()
    set_data = set([tuple(i) for i in data]) 
    set_A=set_data
    return set_A

def get_set_B(file_path):
    """ get setB"""
    data = pandas.read_table(file_path, sep="\\s+", usecols=["Time", "Cell_K", "Cell_I", "Cell_J"])
    data = data[["Cell_I", "Cell_J", "Cell_K"]].values.tolist()
    set_data = set([tuple(i) for i in data]) 
    return set_data

def save_csv(csv_path,s):
    """Save a collection of flow fields as a csv file """
    data=pandas.DataFrame(data=tuple(s),columns=["Cell_I", "Cell_J", "Cell_K"])
    data.to_csv(csv_path,index=False)

def zlc_save(d,j,b,c,set_A,set_B):
    """ Save flow field calculation results"""
    d=zlc_dir+"/"+d
    if not os.path.exists(d):
        os.mkdir(d)
    save_csv(d+"/有效对流流场.csv",j)
    save_csv(d+"/总对流流场.csv",b)
    save_csv(d+"/无效对流流场.csv",c)
    save_csv(d+"/set_A.csv",set_A)
    save_csv(d+"/set_B.csv",set_B)
def cal_Lv(data_mine,carr):
    Vsum=0
    data_mine=data_mine.tolist()
    set_mine=set([tuple(i) for i in data_mine])
    Vsum_set=set_mine & carr
    Vsum=len(Vsum_set)*62.109
    return Vsum

def calculation_dylc(zj_ijk,set_A,n):
    """ Calculation unit flow field"""
    save_dic=dylc_dir+"/"+n
    if not os.path.exists(save_dic):
        os.mkdir(save_dic)
    for k,v in zj_ijk.items():
        d=save_dic+"/"+k
        if not os.path.exists(d):
            os.mkdir(d)
        set_jiao = set_A & v  # 交集
        set_bing = set_A | v  # 并集
        set_cha = (set_A - v) | (v-set_A)  # 差集
        save_csv(d + "/有效对流流场.csv", set_jiao)
        save_csv(d + "/总对流流场.csv", set_bing)
        save_csv(d + "/无效对流流场.csv", set_cha)

# Multi-objective optimization

In [24]:
mine = pandas.read_table(r'D:\工作需\R程序计算\R程序翻译\0222矿层.txt', sep="\\s+").values.tolist()
arrm = np.array(mine)
class MyProblem(ElementwiseProblem):
    def __init__(self):
        super().__init__(n_var=1,  # Quantity
                         n_obj=2,  # Target
                         n_ieq_constr=2,  # Restrictions
                         xl=np.array([1.0]),  # 流量下限
                         xu=np.array([5.0]),  # 流量上限
                         )

    def _evaluate(self, x, out, *args, **kwargs):
        vol = self.calculation(x)

        '''target value'''
        f1 = x     # Maximize the amount of fluid pumped
        f2 = -vol  # Minimize the external influence range of the flow field

        '''estrictions， <= 0'''
        g1 = x - 10
        g2 = 1 - x

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]

    def calculation(self, q):
        newly_build_dir()
        mpthcell('C13-test', 'C13-test', [q, 2 * q, 4 * q, -4 * q])
        mpthin([q, 2 * q, 4 * q, -4 * q])  # 先执行IN的流场模拟
        print('The pumping volume this time is：%s' % q)
        for i in get_file_list():
            set_A = get_set_A(i[1])
            set_B = get_set_B(i[2])
            set_jiao = set_A & set_B
            arr_real = np.array([i for i in set_jiao])
            Vm = cal_Lv(arrm, set_jiao)
            set_bing = set_A | set_B
            set_cha = (set_A - set_B) | (set_B - set_A)
            zlc_save(i[0], set_jiao, set_bing, set_cha, set_A, set_B)
            volume = len(set_bing) * 62.109 #Calculate the total convection field volume
        return volume
problem = MyProblem()

In [25]:
algorithm = NSGA2(
    pop_size=30,  #Population size (number of first generations)
    n_offsprings=5, #The number of descendants in each subsequent generation
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=0.7, eta=15),
    mutation=PM(prob=0.05, eta=20),
    eliminate_duplicates = True
)
'''Define termination principles'''
termination = get_termination("n_gen", 60)

In [26]:
'''Execute multi-objective optimizer process'''
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

X = res.X
F = res.F # target

本次流量选择为：[3]
266633.93700000003
本次流量选择为：[4]
281229.552
本次流量选择为：[1]
208003.041
本次流量选择为：[2]
241604.01
本次流量选择为：[5]
290359.575
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        5 |      5 |  0.000000E+00 |  0.000000E+00 |             - |             -
本次流量选择为：[2.1146778]
245641.095
本次流量选择为：[1.23511795]
217133.064
本次流量选择为：[3.98948507]
280794.789
本次流量选择为：[2.08628428]
244585.242
     2 |        9 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0255891964 |             f
本次流量选择为：[1.97868615]
240858.70200000002
本次流量选择为：[1.19263304]
215580.339
本次流量选择为：[3.85449039]
279925.26300000004
本次流量选择为：[3.95235406]
280546.353
本次流量选择为：[4.99449539]
290235.357
     3 |       14 |     14 |  0.000000E+00 |  0.000000E+00 |  0.0056646817 |             f
本次流量选择为：[2.08655931]
244212.58800000002
本次流量选择为：[4.95933648]
290111.139
本次流量选择为：[1.94077354]
240051.285
本次流量选择为：[3.03791261]
267130.809
本次流量选择为：[2.02114374]
242225.1
     4 |       19 |     18 |  0.000000E+00 | 

本次流量选择为：[2.83349979]
263280.05100000004
本次流量选择为：[2.1117681]
245020.005
本次流量选择为：[4.98930375]
290421.684
本次流量选择为：[1.93800152]
239678.631
本次流量选择为：[4.04129611]
281788.533
    32 |      159 |     30 |  0.000000E+00 |  0.000000E+00 |  0.0028146826 |             f
本次流量选择为：[1.88384196]
238188.015
本次流量选择为：[3.48240472]
273652.254
本次流量选择为：[1.26903301]
218810.007
本次流量选择为：[1.05143773]
64220.706
本次流量选择为：[4.77097238]
289055.286
    33 |      164 |     30 |  0.000000E+00 |  0.000000E+00 |  0.0010624544 |             f
本次流量选择为：[2.81694183]
263404.26900000003
本次流量选择为：[3.78169819]
278248.32
本次流量选择为：[1.16846471]
214897.14
本次流量选择为：[2.81447945]
263093.724
本次流量选择为：[3.46163902]
273093.273
    34 |      169 |     30 |  0.000000E+00 |  0.000000E+00 |  0.0014759312 |             f
本次流量选择为：[3.09095033]
267441.354
本次流量选择为：[2.09377256]
244212.58800000002
本次流量选择为：[1.87229566]
238125.90600000002
本次流量选择为：[2.97810196]
265950.738
本次流量选择为：[3.93446045]
280608.462
    35 |      174 |     30 |  0.000000E+00 |  0.000000E+00 

In [27]:
print(X)
print(F)
'''save'''
with open(r'D:\工作需\R程序计算\R程序翻译\流量XF结果\X-Q.txt', 'w') as f:
    np.savetxt(f, X, fmt="%s", delimiter="    ")
with open(r'D:\工作需\R程序计算\R程序翻译\流量XF结果\F-Q.txt', 'w') as f:
    np.savetxt(f, F, fmt="%s", delimiter="    ")
'''read'''
arrF=np.array(F)
from sklearn.preprocessing import MinMaxScaler
nF = MinMaxScaler().fit_transform(arrF) 

[[1.        ]
 [4.99599727]
 [2.41242626]
 [2.75365408]
 [3.81326321]
 [4.85122022]
 [3.56508091]
 [4.12609265]
 [2.03282053]
 [1.8157449 ]
 [3.41019829]
 [4.49289212]
 [3.23677106]
 [1.21648957]
 [1.11733804]
 [1.65707645]
 [4.45869155]
 [4.02360486]
 [1.68503299]
 [3.10331328]
 [2.97287625]
 [2.26038519]
 [1.38921485]
 [2.34980939]
 [1.90813033]
 [2.83733402]
 [1.02938406]
 [1.30179909]
 [1.38600853]
 [2.14766346]]
[[ 1.00000000e+00 -2.08003041e+05]
 [ 4.99599727e+00 -2.90608011e+05]
 [ 2.41242626e+00 -2.54025810e+05]
 [ 2.75365408e+00 -2.62099980e+05]
 [ 3.81326321e+00 -2.79179955e+05]
 [ 4.85122022e+00 -2.89303722e+05]
 [ 3.56508091e+00 -2.75329197e+05]
 [ 4.12609265e+00 -2.83217040e+05]
 [ 2.03282053e+00 -2.42721972e+05]
 [ 1.81574490e+00 -2.35765764e+05]
 [ 3.41019829e+00 -2.73341709e+05]
 [ 4.49289212e+00 -2.86198272e+05]
 [ 3.23677106e+00 -2.70236259e+05]
 [ 1.21648957e+00 -2.16636192e+05]
 [ 1.11733804e+00 -2.12474889e+05]
 [ 1.65707645e+00 -2.28747447e+05]
 [ 4.45869155e+00 -

'一些注释需要注意的'

In [57]:
'''Entropy weight method determines the weight of two goals'''
def Entropy(data):    
    P_ij = data / data.sum(axis=0)
    e_ij = (-1 / np.log(data.shape[0])) * P_ij * np.log(P_ij)
    e_ij = np.where(np.isnan(e_ij), 0.0, e_ij)
    return (1 - e_ij.sum(axis=0)) / (1 - e_ij.sum(axis=0)).sum()

weights = np.array([0.5,0.5])
print(weights)

[0.5 0.5]
